## Similarity Search with Hippo and OpenAI

This page discusses the integration of a vector database with OpenAI's Embedding API and OpenAI's Chat API.

We will showcase how OpenAI's Embedding API can be used with our vector database to turn user questions into Hippo query conditions. Then, by conducting a similarity search, we can find text that can answer the user's question. Next, the user's question and text are concatenated into a prompt, which serves as the input for OpenAI's Chat API. Finally, we receive a response from the Chat API offered by OpenAI.

## Getting Started

The only prerequisite here is an API key from the OpenAI website. Make sure you have already started a Hippo instance.

## Installing Dependencies

Initially, we require the installation of certain dependencies, such as OpenAI, Langchain, and Hippo-API. Please note, you should install the appropriate versions tailored to your environment.

In [ ]:
!pip install hippo-api
!pip install langchain==0.0.220
!pip install unstructured==0.9.1
!pip install markdown==3.4.4
!pip install tiktoken==0.4.0
!pip install openai==0.27.8

Note: Python version needs to be >=3.8.

## Best Practice
### Importing Dependency Packages

In [2]:
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import re
from transwarp_hippo_langchain.hippo_langchain import Hippo

### Loading Knowledge Documents

In [3]:
loader = UnstructuredMarkdownLoader('./content/hippo.md')
docs = loader.load()
page_content = re.sub("[\n\r\t]", "", docs[0].page_content)
meta = docs[0].metadata
docs = [Document(page_content=page_content, metadata=meta)]

### Segmenting the Knowledge Document

Here, we use Langchain's RecursiveCharacterTextSplitter for segmentation. The delimiter is a period. After segmentation, the text segment does not exceed 2000 characters, and the number of repeated characters is 200.

In [4]:
text_splitter = RecursiveCharacterTextSplitter(separators='.', chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(docs)

### Declaring the Embedding Model
Below, we create the OpenAI or Azure embedding model using the OpenAIEmbeddings method from Langchain.

In [5]:
#openai
embeddings = OpenAIEmbeddings(openai_api_key="xxx",
                                  model="text-embedding-ada-002")
#azure
# embeddings = OpenAIEmbeddings(
#     openai_api_type="azure",
#     openai_api_base="x x x",
#     openai_api_version="x x x",
#     model="x x x",
#     deployment="x x x",
#     openai_api_key="x x x"
# )

### Declaring Hippo Client

In [6]:
HIPPO_CONNECTION = {
    "host": "ip",
    "port": "port",
    "username": "xxx",
    "password": "xxx"
}

### Storing the Document

In [ ]:
print("input...")

# insert docs
vector_store = Hippo.from_documents(
    docs,
    embedding=embeddings,
    table_name="knowledge_qa",
    connection_args=HIPPO_CONNECTION
)
print("success")

### Conducting Knowledge-based Question and Answer
#### Creating a Large Language Question-Answering Model
Below, we create the OpenAI or Azure large language question-answering model respectively using the AzureChatOpenAI and ChatOpenAI methods from Langchain.

In [8]:
# llm = AzureChatOpenAI(
#     openai_api_base="x x x",
#     openai_api_version="xxx",
#     deployment_name="xxx",
#     openai_api_key="xxx",
#     openai_api_type="azure"
# )

llm = ChatOpenAI(
    openai_api_key="xxx",
    model_name="gpt-3.5-turbo-16k"
)

### Acquiring Related Knowledge Based on the Question：

In [12]:
query = "Please introduce hippo"
# query = "Please introduce Hippo Core Architecture"
# query = "What operations does the Hippo Vector Database support for vector data?"
# query = "Does Hippo use hardware acceleration technology? Briefly introduce hardware acceleration technology."


# Retrieve similar content from the knowledge base,fetch the top two most similar texts.
res = vector_store.similarity_search(query, 2)
content_list = [item.page_content for item in res]
text = ''.join(content_list)

### Constructing a Prompt Template

In [13]:
prompt = f"""
Please use the content of the following [Article] to answer my question. If you don't know, please say you don't know, and the answer should be concise."
[Article]:{text}
Please answer this question in conjunction with the above article:{query}
"""

### Waiting for the Large Language Model to Generate an Answer

In [ ]:
response_with_hippo = llm.predict(prompt)
print(f"response_with_hippo:{response_with_hippo}")
response_without_hippo = llm.predict(query)
print("==========================================")
print(f"response_without_hippo:{response_without_hippo}")